In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
# Lade die Trainingsdaten
train_df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

# Zeige die ersten 5 Zeilen an
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# y_train soll nur die 'label'-Spalte enthalten
y_train = train_df['label']

# X_train soll ALLE Spalten AUSSER der 'label'-Spalte enthalten
# Die .drop()-Methode ist dafür perfekt.
X_train = train_df.drop(columns=['label'])

# Überprüfe die Ergebnisse
print("Shape von y_train:", y_train.shape)
print("Shape von X_train:", X_train.shape)

Shape von y_train: (42000,)
Shape von X_train: (42000, 784)


In [4]:
# WICHTIG: Wir müssen das Gleiche für die Trainings- UND die Testdaten tun!
# Lade zuerst die Testdaten
test_df = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

# 1. Wandle die Pandas DataFrames in NumPy Arrays um
X_train_numpy = X_train.values
X_test_numpy = test_df.values

# 2. Forme die Arrays um (Reshape)
# Die -1 ist ein Platzhalter, der NumPy sagt: "Berechne diese Dimension automatisch für mich."
X_train_reshaped = X_train_numpy.reshape(-1, 28, 28)
X_test_reshaped = X_test_numpy.reshape(-1, 28, 28)

# 3. Überprüfe die neue Form! Das ist der Moment der Wahrheit.
print("Neue Shape von X_train:", X_train_reshaped.shape)
print("Neue Shape von X_test:", X_test_reshaped.shape)

Neue Shape von X_train: (42000, 28, 28)
Neue Shape von X_test: (28000, 28, 28)


In [5]:
# Unsere reshaped Arrays aus dem letzten Schritt
# X_train_reshaped, X_test_reshaped

# 1. Normalisiere die Pixelwerte
X_train_normalized = X_train_reshaped / 255.0
X_test_normalized = X_test_reshaped / 255.0

# 2. Überprüfe das Ergebnis (ein wichtiger Schritt für jeden Profi!)
print("Datentyp nach der Normalisierung:", X_train_normalized.dtype)
print("Maximaler Pixelwert nach der Normalisierung:", X_train_normalized.max())
print("Minimaler Pixelwert nach der Normalisierung:", X_train_normalized.min())

Datentyp nach der Normalisierung: float64
Maximaler Pixelwert nach der Normalisierung: 1.0
Minimaler Pixelwert nach der Normalisierung: 0.0


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

# Ein sehr nützlicher Befehl, um die Architektur zu überprüfen
model.summary()

2025-10-25 13:38:47.877965: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761399528.123195      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761399528.194922      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-10-25 13:39:03.072958: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Fai

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
# Trainiere das Modell
history = model.fit(X_train_normalized, y_train, epochs=5)

Epoch 1/5
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8416 - loss: 0.5506
Epoch 2/5
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9507 - loss: 0.1696
Epoch 3/5
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9629 - loss: 0.1214
Epoch 4/5
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9715 - loss: 0.0933
Epoch 5/5
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9741 - loss: 0.0782


In [9]:
predictions = model.predict(X_test_normalized)

875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [10]:
print ("Shape von Predictions", predictions.shape)

Shape von Predictions (28000, 10)


In [11]:
print("\n Wahrscheinlichkeitsverteilung für das erste Bild:")
print(predictions[0])


 Wahrscheinlichkeitsverteilung für das erste Bild:
[3.4348338e-07 1.7006153e-11 9.9997085e-01 2.6754977e-05 7.4903694e-11
 1.2524113e-07 2.5661000e-09 1.5484940e-06 4.0024602e-07 2.2078424e-09]


In [12]:
print("DEBUG: Starte den finalen Submission-Block.")

# --- SCHRITT A: Entscheiden ---
# Wir überprüfen, ob die 'predictions'-Variable überhaupt existiert
if 'predictions' in locals():
    print(f"DEBUG: 'predictions' existiert. Shape ist: {predictions.shape}")
    final_predictions = np.argmax(predictions, axis=1)
    print(f"DEBUG: np.argmax war erfolgreich. Shape der finalen Predictions ist: {final_predictions.shape}")
else:
    print("FATAL DEBUG ERROR: Die Variable 'predictions' wurde in diesem Lauf nicht erstellt!")
    # Wir erstellen eine Dummy-Variable, damit der Code nicht abstürzt, aber wir wissen, dass etwas falsch ist
    final_predictions = np.zeros(28000, dtype=int)

# --- SCHRITT B: Aufschreiben ---
# Erstelle den DataFrame für die Submission
submission_df = pd.DataFrame({
    "ImageId": range(1, len(final_predictions) + 1),
    "Label": final_predictions
})
print(f"DEBUG: Submission DataFrame erstellt. Er hat {len(submission_df)} Zeilen.")

# Speichere den DataFrame als CSV-Datei
print("DEBUG: Versuche jetzt, 'submission.csv' zu schreiben...")
submission_df.to_csv('submission.csv', index=False)
print("DEBUG: 'submission.csv' wurde hoffentlich erfolgreich geschrieben!")

# Überprüfen, ob die Datei wirklich existiert
!ls -l


DEBUG: Starte den finalen Submission-Block.
DEBUG: 'predictions' existiert. Shape ist: (28000, 10)
DEBUG: np.argmax war erfolgreich. Shape der finalen Predictions ist: (28000,)
DEBUG: Submission DataFrame erstellt. Er hat 28000 Zeilen.
DEBUG: Versuche jetzt, 'submission.csv' zu schreiben...
DEBUG: 'submission.csv' wurde hoffentlich erfolgreich geschrieben!
total 408
---------- 1 root root 201240 Oct 25 13:39 __notebook__.ipynb
-rw-r--r-- 1 root root 212908 Oct 25 13:39 submission.csv
